In [4]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import * 
from fastai.vision import *
from fastai.tabular.all import *
from numpy import random
import pandas as pd
import numpy as np
import array as arr

train_df = pd.read_csv("data/03-05-2021_world.csv")
train_df.head().T
#display_all(pops_raw.head().transpose())

# take the incidence rate / testing rate for the incidence proportion (risk)
# note: ~50% people wear masks.

,0,1,2,3,4
FIPS,NaN,NaN,NaN,NaN,NaN
Admin2,NaN,NaN,NaN,NaN,NaN
Province_State,NaN,NaN,NaN,NaN,NaN
Country_Region,Afghanistan,Albania,Algeria,Andorra,Angola
Last_Update,2021-03-06 04:23:41,2021-03-06 04:23:41,2021-03-06 04:23:41,2021-03-06 04:23:41,2021-03-06 04:23:41
Lat,33.93911,41.1533,28.0339,42.5063,-11.2027
Long_,67.709953,20.1683,1.6596,1.5218,17.8739
Confirmed,55827,111301,113948,10998,21026
Deaths,2449,1897,3007,112,511
Recovered,49362,73610,78824,10584,19624


In [5]:
# take care of missing values
train_df['Incident_Rate'].fillna(0, inplace=True)
display_all(train_df.isnull().sum().sort_index()/len(train_df))
#train_df.fillna(0, inplace=True)

Active                 0.001005
Admin2                 0.178850
Case_Fatality_Ratio    0.009797
Combined_Key           0.000000
Confirmed              0.000000
Country_Region         0.000000
Deaths                 0.000000
FIPS                   0.180106
Incident_Rate          0.000000
Last_Update            0.000000
Lat                    0.022105
Long_                  0.022105
Province_State         0.042954
Recovered              0.000000
dtype: float64

In [6]:
train_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-03-06 04:23:41,33.93911,67.709953,55827,2449,49362,4016.0,Afghanistan,143.409656,4.386766
1,NaN,NaN,NaN,Albania,2021-03-06 04:23:41,41.15330,20.168300,111301,1897,73610,35794.0,Albania,3867.572451,1.704387
2,NaN,NaN,NaN,Algeria,2021-03-06 04:23:41,28.03390,1.659600,113948,3007,78824,32117.0,Algeria,259.852428,2.638923
3,NaN,NaN,NaN,Andorra,2021-03-06 04:23:41,42.50630,1.521800,10998,112,10584,302.0,Andorra,14234.129295,1.018367
4,NaN,NaN,NaN,Angola,2021-03-06 04:23:41,-11.20270,17.873900,21026,511,19624,891.0,Angola,63.974407,2.430324


In [7]:
#idx = np.random.permutation(range(n))[:2000]
#idx.sort()
#small_train_df = train_df.iloc[idx[:1000]]
#small_test_df = train_df.iloc[idx[1000:]]
small_cont_vars = ['Active', 'Confirmed', 'Deaths', 'Recovered', 'Case_Fatality_Ratio']
small_cat_vars =  ['Province_State', 'Last_Update', 'Admin2', 'FIPS']
#small_train_df = small_train_df[small_cat_vars + small_cont_vars + ['Incident_Rate']]
#small_test_df = small_test_df[small_cat_vars + small_cont_vars + ['Incident_Rate']]
procs = [Categorify, FillMissing, Normalize]

In [8]:
#small_train_df.head()

In [9]:
#small_test_df.head()

In [10]:
dls = TabularDataLoaders.from_df(train_df, path = "data/03-05-2021_world.csv", procs=procs, cat_names=small_cat_vars, cont_names=small_cont_vars, y_names = 'Incident_Rate', 
                                  valid_idx=list(range(800,1000)), bs=64)

In [11]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

In [12]:
to = TabularPandas(train_df, procs=procs,
                   cat_names = small_cat_vars,
                   cont_names = small_cont_vars,
                   y_names = 'Incident_Rate',
                   splits=splits)

In [13]:
to.xs.iloc[:3]

,Province_State,Last_Update,Admin2,FIPS,Active_na,Case_Fatality_Ratio_na,Active,Confirmed,Deaths,Recovered,Case_Fatality_Ratio
2037,311,7,960,1077,1,1,-0.128009,-0.198407,-0.173755,-0.143911,0.185139
1484,203,7,375,636,1,1,-0.077665,-0.167314,-0.155054,-0.143911,-0.155573
1200,154,7,1816,426,1,1,-0.082839,-0.169980,-0.136352,-0.143911,0.545878


In [14]:
dls = to.dataloaders(bs=64)

In [15]:
dls.show_batch()

,Province_State,Last_Update,Admin2,FIPS,Active_na,Case_Fatality_Ratio_na,Active,Confirmed,Deaths,Recovered,Case_Fatality_Ratio,Incident_Rate
0,Hyogo,2021-03-06 04:23:41,#na#,#na#,False,False,462.999595,18190.999779,555.000003,17172.999949,3.050959,332.791199
1,Kansas,2021-03-06 04:23:41,Wichita,20203.0,False,False,209.000238,214.999261,6.000004,0.000018,2.790698,10146.295898
2,Missouri,2021-03-06 04:23:41,Phelps,29161.0,False,False,2911.000228,3032.000768,120.999989,0.000018,3.990765,6802.324219
3,New Mexico,2021-03-06 04:23:41,Valencia,35061.0,False,False,6101.999795,6200.999554,99.000013,0.000018,1.596517,8086.010742
4,Michigan,2021-03-06 04:23:41,Alcona,26001.0,False,False,406.999776,430.999036,23.999989,0.000018,5.568446,4142.239258
5,Baden-Wurttemberg,2021-03-06 04:23:41,#na#,#na#,False,False,12896.999930,322272.012933,8251.000139,301124.002832,2.560260,2911.342285
6,South Carolina,2021-03-06 04:23:41,Newberry,45071.0,False,False,4331.999997,4424.000912,91.999990,0.000018,2.079566,11508.844727
7,Tennessee,2021-03-06 04:23:41,Houston,47083.0,False,False,1004.999979,1037.000911,32.000023,0.000018,3.085824,12644.799805
8,Illinois,2021-03-06 04:23:41,Knox,17095.0,False,False,4888.999799,5021.999074,133.000014,0.000018,2.648347,10104.831055
9,Oklahoma,2021-03-06 04:23:41,Canadian,40017.0,False,False,15929.000033,16026.000479,96.999992,0.000018,0.605266,10806.036133


In [16]:
learn = tabular_learner(dls, metrics=accuracy)

In [17]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,73631288.000000,74657664.000000,0.020101,00:01


In [18]:
learn.show_results()

,Province_State,Last_Update,Admin2,FIPS,Active_na,Case_Fatality_Ratio_na,Active,Confirmed,Deaths,Recovered,Case_Fatality_Ratio,Incident_Rate,Incident_Rate_pred
0,496.0,7.0,963.0,0.0,1.0,1.0,-0.113920,-0.189397,-0.156470,-0.143911,0.891087,15372.682617,0.023769
1,496.0,7.0,938.0,0.0,1.0,1.0,-0.133805,-0.201953,-0.174605,-0.143911,1.759784,6932.314453,0.022441
2,408.0,7.0,808.0,0.0,1.0,1.0,-0.133040,-0.201597,-0.178856,-0.143911,-0.912646,1952.571167,0.007312
3,203.0,7.0,1893.0,0.0,1.0,1.0,-0.128786,-0.198952,-0.176589,-0.143911,-0.369880,9036.715820,0.017852
4,141.0,7.0,1315.0,0.0,1.0,1.0,1.299682,0.687870,0.532371,-0.143911,0.013002,8198.253906,0.034282
5,357.0,7.0,1018.0,0.0,1.0,1.0,-0.134139,-0.202208,-0.176589,-0.143911,0.740437,11837.837891,0.020768
6,154.0,7.0,1726.0,0.0,1.0,1.0,-0.099533,-0.180466,-0.149387,-0.143911,0.553043,11967.946289,0.023187
7,494.0,7.0,286.0,0.0,1.0,1.0,-0.066862,-0.160229,-0.134935,-0.143911,0.257173,10632.902344,0.032065
8,526.0,7.0,0.0,0.0,1.0,1.0,-0.130017,0.503126,0.301152,0.711983,-0.124664,864.689636,0.066603


In [16]:
preds, targs = learn.get_preds()

In [17]:
cum_prob = (1.8+2.1)/100; cum_prob # probability according to: https://www.acpjournals.org/doi/10.7326/M20-6817

0.03900000000000001

In [ ]:
prob_mask = 1.8/3.9; prob_mask

In [ ]:
prob_nomask = 2.1/3.9; prob_nomask

In [ ]:
irlen = len(train_df.Incident_Rate);

In [ ]:
train_df.Incident_Rate
infected_mask_list = [0] * irlen
country_list = [0] * irlen

In [ ]:
np.int64(np.ceil(train_df.Incident_Rate[1]))

In [ ]:
for i in range(irlen):
    i_rate = np.int64(np.ceil(train_df.Incident_Rate[i]))
    country_list[i] = train_df.Country_Region[i]
    imask_counter = 0
    for j in range(i_rate):
        x = random.rand()
        if x <= prob_mask:
            imask_counter += 1
    infected_mask_list[i] = imask_counter

In [ ]:
for k in range(irlen):
    display(f'Country: {country_list[k]}, with {infected_mask_list[k]} wearing masks out of {np.int64(np.ceil(train_df.Incident_Rate[k]))}') # display the array, sorted by region/country alphabetically. each entry is how many cases were infected while wearing masks

'Country: Afghanistan, with 71 wearing masks out of 144'

'Country: Albania, with 1846 wearing masks out of 3868'

'Country: Algeria, with 131 wearing masks out of 260'

'Country: Andorra, with 6587 wearing masks out of 14235'

'Country: Angola, with 31 wearing masks out of 64'

...

'Country: US, with 2248 wearing masks out of 4867'

'Country: US, with 4203 wearing masks out of 9085'

'Country: US, with 3688 wearing masks out of 7897'

'Country: US, with 4869 wearing masks out of 10584'

'Country: US, with 2824 wearing masks out of 6197'